In [ ]:
load '../../../global/datasets/Ionosphere.py'

In [12]:
# %load '../../../global/samplers/RandomTrainTestSampler.py'
import torch
import random

class SubsetSampler(object):

    def __init__(self, subset):
        self.subset = subset

    def __iter__(self):
        return iter(self.subset)

    def __len__(self):
        return len(self.subset)


class RandomTrainTestSampler(object):

    def __init__(self, data_source, train_share=0.8):

        # Generate a list of indizes reaching from 0 ... len(data_source)-1
        idxList = list(range(0,len(data_source)))

        # Ensure that list is sorted randomly
        random.shuffle(idxList)

        # Split dataset random shares of train and test data
        numberOfTrainSamples = int(len(data_source) / (1 / train_share))
        
        self.train_samples = idxList[:numberOfTrainSamples]
        self.test_samples = idxList[numberOfTrainSamples:]

    def trainSampler(self):
        return SubsetSampler(self.train_samples)

    def testSampler(self):
        return SubsetSampler(self.test_samples)


In [13]:
import numpy as np
import torch

class IncreaseDimension(object):

    def __call__(self, o):

        return np.expand_dims(o, axis=0)


In [14]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):

        super(Net, self).__init__()

        self.features = nn.Sequential(
            nn.Conv1d(1, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 16, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 2),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 16)
        x = self.classifier(x)
        return x

In [ ]:
import torch
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import torchvision.datasets
import torchvision.models as models

transformsData = transforms.Compose([IncreaseDimension()] )

dataset = Ionosphere(root='./data', transform=transformsData, download=True)

sampler = RandomTrainTestSampler(dataset)

trainloader = torch.utils.data.DataLoader(dataset, batch_size=20, sampler=sampler.trainSampler(), num_workers=2 )
testloader = torch.utils.data.DataLoader(dataset, batch_size=20, sampler=sampler.testSampler(), num_workers=2)

net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.0)

print("Start training")
for epoch in range(300):
    
    print("Epoch: ", epoch)
    
    #if epoch == 40:
    #    optimizer = optim.SGD(net.parameters(), lr=0.000001, momentum=0.9)

    running_loss = 0.0
    for data in trainloader:

        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)
  
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


    print(loss.data[0])

    correct = 0
    total = 0
    
    for data in testloader:
  
        inputs, labels = data
        outputs = net(Variable(inputs))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        
    print('Accuracy of the network on the test images: %f %%' % (100 * correct / total))

print('Finished Training')




Files already downloaded and verified
Start training
Epoch:  0
0.6751001477241516
Accuracy of the network on the test images: 66.197183 %
Epoch:  1
0.6614413857460022
Accuracy of the network on the test images: 66.197183 %
Epoch:  2
0.6470427513122559
Accuracy of the network on the test images: 66.197183 %
Epoch:  3
0.6476805806159973
Accuracy of the network on the test images: 66.197183 %
Epoch:  4
0.6371049880981445
Accuracy of the network on the test images: 66.197183 %
Epoch:  5
0.6208263039588928
Accuracy of the network on the test images: 66.197183 %
Epoch:  6
0.6156182289123535
Accuracy of the network on the test images: 66.197183 %
Epoch:  7
0.6078538298606873
Accuracy of the network on the test images: 66.197183 %
Epoch:  8
0.6018950939178467
Accuracy of the network on the test images: 66.197183 %
Epoch:  9
0.5893607139587402
Accuracy of the network on the test images: 66.197183 %
Epoch:  10
0.5813437104225159
Accuracy of the network on the test images: 66.197183 %
Epoch:  11


Accuracy of the network on the test images: 92.957746 %
Epoch:  95
0.08222819864749908
Accuracy of the network on the test images: 94.366197 %
Epoch:  96
0.0951085016131401
Accuracy of the network on the test images: 91.549296 %
Epoch:  97
0.10523412376642227
Accuracy of the network on the test images: 92.957746 %
Epoch:  98
0.1000637412071228
Accuracy of the network on the test images: 91.549296 %
Epoch:  99
0.09873814880847931
Accuracy of the network on the test images: 94.366197 %
Epoch:  100
0.13233602046966553
Accuracy of the network on the test images: 92.957746 %
Epoch:  101
